## World Development Indicators - Analytics

In [47]:
import pandas as pd
import numpy as np
import time

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = datos['CountryCode'].drop_duplicates().values.tolist()

preprocessing = 'imput'
search = 'original'
year_init = 2012
year_range = 5
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura = open("Indicadores/iagricultura.txt").read().split(',')
economia    = open("Indicadores/ieconomia.txt").read().split(',')
educacion   = open("Indicadores/ieducacion.txt").read().split(',')
salud       = open("Indicadores/isalud.txt").read().split(',')
deuda       = open("Indicadores/ideuda.txt").read().split(',')

conjunto_nombre = ['Agricultura','Economia','Educacion','Salud','Deuda']
conjunto = [agricultura,economia,educacion,salud,deuda]
datos

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Antigua and Barbuda,ATG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.261440e+02
1,Antigua and Barbuda,ATG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.823712e+01
2,Antigua and Barbuda,ATG,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,7.779958e+00
3,Antigua and Barbuda,ATG,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.045716e+01
4,Antigua and Barbuda,ATG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,3.292000e+01
5,Antigua and Barbuda,ATG,CO2 emissions (kt),EN.ATM.CO2E.KT,1960,3.667000e+01
6,Antigua and Barbuda,ATG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,1960,6.706169e-01
7,Antigua and Barbuda,ATG,CO2 emissions from gaseous fuel consumption (%...,EN.ATM.CO2E.GF.ZS,1960,0.000000e+00
8,Antigua and Barbuda,ATG,CO2 emissions from gaseous fuel consumption (kt),EN.ATM.CO2E.GF.KT,1960,0.000000e+00
9,Antigua and Barbuda,ATG,CO2 emissions from liquid fuel consumption (% ...,EN.ATM.CO2E.LF.ZS,1960,1.000000e+02


In [53]:
def r2(y_true, y_predict):
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_predict)

In [54]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [55]:
def tabla_2_base(indicadores,look_back):
    temp_table = []
    for i in range(look_back):      
        temp_years = range(year_init-year_range-i+1,year_init-i+1)[::-1]
        temp_table.append(pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,temp_years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False]))
    return pd.DataFrame(np.column_stack(temp_table))

In [56]:
def estimator_Universal(estimador, X_train, X_test, y_train, y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVR
    
    if(search=='original'):
        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor()
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1)
        if(estimador=='SVR'):
            estimator = SVR()
    else:
        best_params = SearchCV_Universal(estimador, search, X_train, y_train)       

        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor().set_params(**best_params)
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
        if(estimador=='SVR'):
            estimator = SVR().set_params(**best_params)
        
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
        
    return r2(y_test,y_predict)

In [57]:
def Preprocess(tab1,tab2,y_indicator):
    from sklearn.preprocessing import Imputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
       
    #Fusionamos la tabla_1 y el indicador y de la tabla_2
    tab_fusion = np.column_stack((np.array(tab1)[:,:],np.array(tab2)[:,y_indicator]))    
    
    #Eliminamos las columnas de NaN descartando Indicadores que no tienen regristros para ningún pais y año deseados
    df = pd.DataFrame(tab_fusion)
    df = df.dropna(how='all',axis=1)
    
    #Eliminamos las filas Si el valor a predecir es NaN
    df = df.dropna(subset=[df.iloc[:,-1].name])
    
    if(preprocessing=='zeros'):
        #Imputamos los NaN por Zero
        df = df.fillna(0)      
        
    if(preprocessing=='imput'):
        #Imputamos los NaN por la media de cada Indicador respectivamente      
        impute=Imputer(missing_values="NaN",strategy='mean',axis=0)
        impute.fit(df)
        df = pd.DataFrame(impute.transform(df))
        
    # Asignamos X e y, eliminando los indicadores que se correlacionen más (coeficiente > 0.7)
    df_ = df.corr()
    X = df.drop(df_[df_.iloc[:,-1] > 0.7].index, axis=1)
    y = df.iloc[:,-1]

    # Normalizamos los datos
    sc = StandardScaler()
    df_norm = sc.fit_transform(np.column_stack([X,y]))
    X = df_norm[:,:-1]
    y = df_norm[:,-1]
    
    # Separamos Train y Test respectivamente para X e y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

In [58]:
def iter_Splitter_Optimus(tab1,tab2): 
    R2_global = list()
    for i in range(0,np.shape(tab2.dropna(how='all',axis=1))[1]):

        X_train, X_test, y_train, y_test = Preprocess(tab1, tab2, i)

        result = estimator_Universal('DTR', X_train, X_test, y_train, y_test)

        if(result < 0.9): 
            temp = estimator_Universal('SVR', X_train, X_test, y_train, y_test)
            if(temp < 0.9): 
                temp2 = estimator_Universal('RFR', X_train, X_test, y_train, y_test)
                if (temp2 > temp): 
                    result = temp2
            if(temp > result): 
                result = temp

        R2_global.append(result)
    return R2_global

In [59]:
def SearchCV_Universal(estimador, search, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import ShuffleSplit
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.svm import SVR
        
    if(estimador=='DTR'):
        estimator  = DecisionTreeRegressor()
        param_grid = {  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                        'splitter': ['best', 'random']
                     }
        
    if(estimador=='RFR'):
        estimator  = RandomForestRegressor()       
        param_grid = { 
                        "n_estimators"      : [10,20,30,40],
                        "max_features"      : ["auto", "sqrt", "log2"],
                        "min_samples_split" : [2,4,8],
                        "bootstrap": [True, False],
                     }
    if(estimador=='SVR'):
        estimator  = SVR()
        param_grid ={
                        'gamma'  : ['auto', 1e-3, 1e-4],
                        'C'      : [1, 10, 100, 1000],            
                    }

    if (search=='random'):
        grid = RandomizedSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
    if (search=='grid'):
        grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
        
    grid.fit(X_train, y_train)
    
    return grid.best_params_

In [98]:
indicadores_1 = salud
indicadores_2 = deuda

if(indicadores_1!=indicadores_2):
    df = pd.DataFrame(indicadores_2)
    indicadores_2 = np.array(df.loc[~df.ix[:,0].isin(indicadores_1)]).flatten()

tabla_base(indicadores_2)

IndicatorCode     DT.GPA.DPPG  DT.GPA.OFFT  DT.GPA.PRVT  DT.GRE.DPPG  \
CountryCode Year                                                       
ABW         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
ARG         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
ATG         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
BHS         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
BLZ         2012       3.0211       3.0211       0.0000      66.9211   
            2011       5.0858       5.0858       0.0000      45.3851   
            2010       5.5306       5.5306       0.0000      55.3044   
            2009       4.5687       4.5687       0.0000      54.6719   
            2008       5.9389       5.3372      12.5000      40.7730   
BOL         2012       8.1974       6.6735      10.0000      44.9126   
            2011       1.1045       1.1045       0.0000      50.7465   
            2010       6.0631       4.6521      26.7759      54.2757   
            2009       4.9990       5.2001       4.5000      45.6675   
            2008       5.1230       5.3313       2.0000      39.9502   
...                       ...          ...          ...          ...   
TCA         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
TTO         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
URY         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          NaN          NaN          NaN   
            2009          NaN          NaN          NaN          NaN   
            2008          NaN          NaN          NaN          NaN   
VCT         2012       5.5517       5.5517       0.0000      58.3196   
            2011       1.8097       1.8097       0.0000      58.1308   
            2010       4.1661       4.1661       0.0000      46.6072   
            2009       5.3685       5.3685       0.0000      48.2134   
            2008       0.5000       0.0000       0.5000       4.7895   
VEN         2012          NaN          NaN          NaN          NaN   
            2011          NaN          NaN          NaN          NaN   
            2010          NaN          N

In [24]:
def iterador_global(indicadores_1,indicadores_2,look_back): 
    
    if(indicadores_1!=indicadores_2):
        df = pd.DataFrame(indicadores_2)
        indicadores_2 = np.array(df.loc[~df.ix[:,0].isin(indicadores_1)]).flatten()
    
    tab1 = tabla_2_base(indicadores_1,look_back)
    tab2 = tabla_base(indicadores_2)
    
    return iter_Splitter_Optimus(tab1,tab2)

In [25]:
def porcent_result(df):
    df_temp = df
    x = 0.0
    for i in range(len(df_temp)):
        if(df_temp[i] > 0.9):
            x=x+1
    porcentaje = x/len(df)
    df = pd.DataFrame(df)
    df[df < 0] = 0.0
    return porcentaje , np.array(df.values)

In [26]:
def resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]
    
    # Códigos de los indicadores con registros    
    for i in range(len(conjunto_nombre)):
        if indicador_2 == conjunto_nombre[i]:
            df_zeros = tabla_base(conjunto[i]).dropna(how='all',axis=1)
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,temp]),columns=['Indicadores','Resultados'])
    
    return df_results

In [27]:
def buscar_mejores(df):
    return df[df.Resultados > 0.9].reset_index(drop=True)

In [ ]:
start_time = time.time()
resultado_global = []
log_detail = []

for i in range(len(conjunto)):
    for j in range(len(conjunto)):
        for k in range(1,look_back+1):
            start_time2 = time.time()
            porcent , results = porcent_result(iterador_global(conjunto[i],conjunto[j],k))
            resultado_global.append([conjunto_nombre[i],conjunto_nombre[j],k,porcent,"%s" % (time.time() - start_time2),results])
            pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","%","Time","Results"]).to_csv('log_global.csv')

df_global = pd.DataFrame(resultado_base, columns=["Base","Target","Look Back","%","Time","Results"])
print("--- %s seconds ---" % (time.time() - start_time))
df_global

In [38]:
start_time = time.time()
resultado_base = []
log_detail = []

for i in range(len(conjunto)):
    for j in range(1,look_back+1):
        start_time2 = time.time()
        porcent , results = porcent_result(iterador_global(conjunto[0],conjunto[i],j))
        resultado_base.append([conjunto_nombre[0],conjunto_nombre[i],j,porcent,"%s" % (time.time() - start_time2),results])
        pd.DataFrame(resultado_base, columns=["Base","Target","Look Back","%","Time","Results"]).to_csv('log_base.csv')

df = pd.DataFrame(resultado_base, columns=["Base","Target","Look Back","%","Time","Results"])
print("--- %s seconds ---" % (time.time() - start_time))
df

--- 593.727627039 seconds ---


,Base,Target,Look Back,%,Time,Results
0,Agricultura,Agricultura,1,0.409091,11.7917878628,"[[0.743257252517], [0.799267160063], [0.450469..."
1,Agricultura,Agricultura,2,0.340909,13.7249598503,"[[0.187981863047], [0.874508100624], [0.764390..."
2,Agricultura,Agricultura,3,0.477273,14.5300331116,"[[0.812738662974], [0.814295907853], [0.494452..."
3,Agricultura,Economia,1,0.448413,51.2642700672,"[[0.979029008353], [0.99908188219], [0.0], [0...."
4,Agricultura,Economia,2,0.365079,60.8727319241,"[[0.954718270994], [0.99237330478], [0.0], [0...."
5,Agricultura,Economia,3,0.365079,61.9855110645,"[[0.990165691669], [0.97183304533], [0.0], [0...."
6,Agricultura,Educacion,1,0.396552,26.1679749489,"[[0.587949512639], [0.647628477597], [0.651698..."
7,Agricultura,Educacion,2,0.224138,33.4832720757,"[[0.611922307604], [0.633514832122], [0.539905..."
8,Agricultura,Educacion,3,0.163793,37.0415539742,"[[0.578652314202], [0.36039847253], [0.4306014..."
9,Agricultura,Salud,1,0.417323,27.8954219818,"[[0.336245994132], [0.960292067484], [0.830765..."


In [40]:
pd.DataFrame.pivot_table(df, index=["Base","Target"], columns=["Look Back"])

%                    
Look Back                       1         2         3
Base        Target                                   
Agricultura Agricultura  0.409091  0.340909  0.477273
            Deuda        0.294643  0.232143  0.209821
            Economia     0.448413  0.365079  0.365079
            Educacion    0.396552  0.224138  0.163793
            Salud        0.417323  0.307087  0.307087

In [65]:
buscar_mejores(resultados(df,'Agricultura','Economia',2))

,Indicadores,Resultados
0,Adjusted net national income (annual % growth),0.954718
1,Adjusted net national income (constant 2005 US$),0.992373
2,Adjusted net national income per capita (annua...,0.990908
3,"Adjusted net savings, excluding particulate em...",0.952358
4,Adjusted savings: consumption of fixed capital...,0.977105
5,Adjusted savings: consumption of fixed capital...,0.977951
6,Adjusted savings: education expenditure (% of ...,0.946808
7,Adjusted savings: mineral depletion (% of GNI),0.941409
8,Adjusted savings: net forest depletion (% of GNI),0.956653
9,Adjusted savings: net national savings (curren...,0.9313
